# Лабораторная работа №5. Линейные методы. Задание

In [1]:
import pickle
import numpy as np
import pandas as pd
from tqdm import tqdm_notebook
from scipy.sparse import csr_matrix, hstack
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LogisticRegression
%matplotlib inline
from matplotlib import pyplot as plt
import seaborn as sns

C:\Users\Lapte\AppData\Local\Temp\ipykernel_18768\2678303591.py:3: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd
C:\Users\Lapte\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


### 1. Загрузка и преобразование данных
Зарегистрируйтесь на [Kaggle](www.kaggle.com), если вы не сделали этого раньше, зайдите на [страницу](https://inclass.kaggle.com/c/catch-me-if-you-can-intruder-detection-through-webpage-session-tracking2) соревнования и скачайте данные. Первым делом загрузим обучающую и тестовую выборки и посмотрим на данные.

In [2]:
# загрузим обучающую и тестовую выборки
train_df = pd.read_csv('./data/train_sessions.csv',
                       index_col='session_id')
test_df = pd.read_csv('./data/test_sessions.csv',
                      index_col='session_id')

# приведем колонки time1, ..., time10 к временному формату
times = ['time%s' % i for i in range(1, 11)]
train_df[times] = train_df[times].apply(pd.to_datetime)
test_df[times] = test_df[times].apply(pd.to_datetime)

# отсортируем данные по времени
train_df = train_df.sort_values(by='time1')

# посмотрим на заголовок обучающей выборки
train_df.head()

,site1,time1,site2,time2,site3,time3,site4,time4,site5,time5,...,time6,site7,time7,site8,time8,site9,time9,site10,time10,target
session_id,,,,,,,,,,,,,,,,,,,,,
21669,56,2013-01-12 08:05:57,55.0,2013-01-12 08:05:57,NaN,NaT,NaN,NaT,NaN,NaT,...,NaT,NaN,NaT,NaN,NaT,NaN,NaT,NaN,NaT,0
54843,56,2013-01-12 08:37:23,55.0,2013-01-12 08:37:23,56.0,2013-01-12 09:07:07,55.0,2013-01-12 09:07:09,NaN,NaT,...,NaT,NaN,NaT,NaN,NaT,NaN,NaT,NaN,NaT,0
77292,946,2013-01-12 08:50:13,946.0,2013-01-12 08:50:14,951.0,2013-01-12 08:50:15,946.0,2013-01-12 08:50:15,946.0,2013-01-12 08:50:16,...,2013-01-12 08:50:16,948.0,2013-01-12 08:50:16,784.0,2013-01-12 08:50:16,949.0,2013-01-12 08:50:17,946.0,2013-01-12 08:50:17,0
114021,945,2013-01-12 08:50:17,948.0,2013-01-12 08:50:17,949.0,2013-01-12 08:50:18,948.0,2013-01-12 08:50:18,945.0,2013-01-12 08:50:18,...,2013-01-12 08:50:18,947.0,2013-01-12 08:50:19,945.0,2013-01-12 08:50:19,946.0,2013-01-12 08:50:19,946.0,2013-01-12 08:50:20,0
146670,947,2013-01-12 08:50:20,950.0,2013-01-12 08:50:20,948.0,2013-01-12 08:50:20,947.0,2013-01-12 08:50:21,950.0,2013-01-12 08:50:21,...,2013-01-12 08:50:21,946.0,2013-01-12 08:50:21,951.0,2013-01-12 08:50:22,946.0,2013-01-12 08:50:22,947.0,2013-01-12 08:50:22,0


В обучающей выборке содержатся следующие признаки:
    - site1 – индекс первого посещенного сайта в сессии
    - time1 – время посещения первого сайта в сессии
    - ...
    - site10 – индекс 10-го посещенного сайта в сессии
    - time10 – время посещения 10-го сайта в сессии
    - target – целевая переменная, 1 для сессий Элис, 0 для сессий других пользователей
    
Сессии пользователей выделены таким образом, что они не могут быть длиннее получаса или 10 сайтов. То есть сессия считается оконченной либо когда пользователь посетил 10 сайтов подряд либо когда сессия заняла по времени более 30 минут.

В таблице встречаются пропущенные значения, это значит, что сессия состоит менее, чем из 10 сайтов. Заменим пропущенные значения нулями и приведем признаки к целому типу. Также загрузим словарь сайтов и посмотрим, как он выглядит:

In [3]:
# приведем колонки site1, ..., site10 к целочисленному формату и заменим пропуски нулями
sites = ['site%s' % i for i in range(1, 11)]
train_df[sites] = train_df[sites].fillna(0).astype('int')
test_df[sites] = test_df[sites].fillna(0).astype('int')

# загрузим словарик сайтов
with open(r"./data/site_dic.pkl", "rb") as input_file:
    site_dict = pickle.load(input_file)

# датафрейм словарика сайтов
sites_dict_df = pd.DataFrame(list(site_dict.keys()), 
                          index=list(site_dict.values()), 
                          columns=['site'])
print(u'всего сайтов:', sites_dict_df.shape[0])
sites_dict_df.head()

всего сайтов: 48371


,site
25075,www.abmecatronique.com
13997,groups.live.com
42436,majeureliguefootball.wordpress.com
30911,cdt46.media.tourinsoft.eu
8104,www.hdwallpapers.eu


Выделим целевую переменную и объединим выборки, чтобы вместе привести их к разреженному формату.

In [4]:
# наша целевая переменная
y_train = train_df['target']

# объединенная таблица исходных данных
full_df = pd.concat([train_df.drop('target', axis=1), test_df])

# индекс, по которому будем отделять обучающую выборку от тестовой
idx_split = train_df.shape[0]

Для самой первой модели будем использовать только посещенные сайты в сессии (но не будем обращать внимание на временные признаки). За таким выбором данных для модели стоит такая идея:  *у Элис есть свои излюбленные сайты, и чем чаще вы видим эти сайты в сессии, тем выше вероятность, что это сессия Элис и наоборот.*

Подготовим данные, из всей таблицы выберем только признаки `site1, site2, ... , site10`. Напомним, что пропущенные значения заменены нулем. Вот как выглядят первые строки таблицы:

In [5]:
# табличка с индексами посещенных сайтов в сессии
full_sites = full_df[sites]
full_sites.head()

,site1,site2,site3,site4,site5,site6,site7,site8,site9,site10
session_id,,,,,,,,,,
21669,56,55,0,0,0,0,0,0,0,0
54843,56,55,56,55,0,0,0,0,0,0
77292,946,946,951,946,946,945,948,784,949,946
114021,945,948,949,948,945,946,947,945,946,946
146670,947,950,948,947,950,952,946,951,946,947


Сессии представляют собой последовательность индексов сайтов и данные в таком виде неудобны для линейных методов. В соответствии с нашей гипотезой (у Элис есть излюбленные сайты) надо преобразовать эту таблицу таким образом, чтобы каждому возможному сайту соответствовал свой отдельный признак (колонка), а его значение равнялось бы количеству посещений этого сайта в сессии. Это делается в две строчки:

In [6]:
from scipy.sparse import csr_matrix

In [ ]:
csr_matrix?

In [8]:
# последовательность с индексами
sites_flatten = full_sites.values.flatten()

# искомая матрица
full_sites_sparse = csr_matrix(([1] * sites_flatten.shape[0],
                                sites_flatten,
                                range(0, sites_flatten.shape[0] + 10, 10)))[:, 1:]

Еще один плюс использования разреженных матриц в том, что для них имеются специальные реализации как матричных операций, так и алгоритмов машинного обучения, что подчас позволяет ощутимо ускорить операции за счет особенностей структуры данных. Это касается и логистической регрессии. Вот теперь у нас все готово для построения нашей первой модели.

### 2. Построение первой модели

Итак, у нас есть алгоритм и данные для него, построим нашу первую модель, воспользовавшись релизацией [логистической регрессии](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html) из пакета `sklearn` с параметрами по умолчанию. Первые 90% данных будем использовать для обучения (обучающая выборка отсортирована по времени), а оставшиеся 10% для проверки качества (validation). 

**Напишите простую функцию, которая будет возвращать качество модели на отложенной выборке, и обучите наш первый классификатор**.

In [9]:
from sklearn.model_selection import train_test_split

def get_auc_lr_valid(X, y, C=1.0, ratio = 0.9, seed=17):
    '''
    X, y – выборка
    ratio – в каком отношении поделить выборку
    C, seed – коэф-т регуляризации и random_state 
              логистической регрессии
    '''
    # Разбиваем выборку на обучающую и валидационную
    X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=(1 - ratio), random_state=seed)

    logit = LogisticRegression(C=C, random_state=seed, solver='liblinear')
    logit.fit(X_train, y_train)

    # Получаем предсказания вероятностей для валидационной выборки
    valid_predict_proba = logit.predict_proba(X_valid)[:, 1]

    # Вычисляем ROC AUC
    auc = roc_auc_score(y_valid, valid_predict_proba)
    
    return auc
    # Ваш код здесь

**Посмотрите, какой получился ROC AUC на отложенной выборке.**

In [10]:
# Ваш код здесь
# Используем наши данные и обучаем модель
roc_auc = get_auc_lr_valid(full_sites_sparse[:idx_split, :], y_train)
print(f'ROC AUC на валидационной выборке: {roc_auc}')

ROC AUC на валидационной выборке: 0.964337527131929


Будем считать эту модель нашей первой отправной точкой (baseline). Для построения модели для прогноза на тестовой выборке **необходимо обучить модель заново уже на всей обучающей выборке** (пока наша модель обучалась лишь на части данных), что повысит ее обобщающую способность:

In [11]:
# Ваш код здесь
# функция для записи прогнозов в файл
def write_to_submission_file(predicted_labels, out_file,
                             target='target', index_label="session_id"):
    predicted_df = pd.DataFrame(predicted_labels,
                                index = np.arange(1, predicted_labels.shape[0] + 1),
                                columns=[target])
    predicted_df.to_csv(out_file, index_label=index_label)

**Обучите модель на всей выборке, сделайте прогноз для тестовой выборки и сделайте посылку в соревновании**.

In [12]:
# Ваш код здесь
logit = LogisticRegression(C=1.0, random_state=17, solver='liblinear')
logit.fit(full_sites_sparse[:idx_split, :], y_train)

# Делаем прогноз для тестовой выборки
test_pred_proba = logit.predict_proba(full_sites_sparse[idx_split:, :])[:, 1]

# Создаем файл для посылки
write_to_submission_file(test_pred_proba, 'baseline_submission.csv')

Если вы выполните эти действия и загрузите ответ на [странице](https://inclass.kaggle.com/c/catch-me-if-you-can-intruder-detection-through-webpage-session-tracking2) соревнования, то воспроизведете первый бенчмарк "Logit".

### 3. Улучшение модели, построение новых признаков

Создайте такой признак, который будет представлять собой число вида ГГГГММ от той даты, когда проходила сессия, например 201407 -- 2014 год и 7 месяц. Таким образом, мы будем учитывать помесячный [линейный тренд](http://people.duke.edu/~rnau/411trend.htm) за весь период предоставленных данных.

In [13]:
# Ваш код здесь
# Создаем новый признак
full_df['start_month'] = full_df['time1'].dt.year * 100 + full_df['time1'].dt.month

Добавьте новый признак, предварительно отмасштабировав его с помощью `StandardScaler`, и снова посчитайте ROC AUC на отложенной выборке.

In [14]:
# Ваш код здесь
# Масштабируем новый признак
scaler = StandardScaler()
full_df['start_month_scaled'] = scaler.fit_transform(full_df['start_month'].values.reshape(-1, 1))

# Разделяем данные обратно на обучающую и тестовую выборки
X_train_scaled = full_df['start_month_scaled'][:idx_split].values.reshape(-1, 1)
X_test_scaled = full_df['start_month_scaled'][idx_split:].values.reshape(-1, 1)

# Вычисляем ROC AUC с использованием нового признака
roc_auc_with_new_feature = get_auc_lr_valid(hstack([full_sites_sparse[:idx_split, :], X_train_scaled]),
                                           y_train)
print(f'ROC AUC на отложенной выборке с новым признаком: {roc_auc}')

ROC AUC на отложенной выборке с новым признаком: 0.964337527131929


**Добавьте два новых признака: start_hour и morning.**

Признак `start_hour` – это час в который началась сессия (от 0 до 23), а бинарный признак `morning` равен 1, если сессия началась утром и 0, если сессия началась позже (будем считать, что утро это если `start_hour равен` 11 или меньше).

**Посчитйте ROC AUC на отложенной выборке для выборки с:**
- сайтами, `start_month` и `start_hour`
- сайтами, `start_month` и `morning`
- сайтами, `start_month`, `start_hour` и `morning`

In [15]:
# Ваш код здесь
# start_hour
full_df['start_hour'] = full_df['time1'].dt.hour
# morning
full_df['morning'] = (full_df['start_hour'] <= 11).astype(int)

# Масштабируем новые признаки
scaler = StandardScaler()
full_df[['start_hour_scaled', 'morning_scaled']] = scaler.fit_transform(full_df[['start_hour', 'morning']])

# Разделяем выборку
X_train_hour_scaled = hstack([full_sites_sparse[:idx_split, :], 
                                       full_df['start_month_scaled'][:idx_split].values.reshape(-1, 1),
                                       full_df['start_hour_scaled'][:idx_split].values.reshape(-1, 1)])
X_train_morning_scaled = hstack([full_sites_sparse[:idx_split, :], 
                                       full_df['start_month_scaled'][:idx_split].values.reshape(-1, 1),
                                       full_df['morning_scaled'][:idx_split].values.reshape(-1, 1)])
X_train_hour_morning_scaled = hstack([full_sites_sparse[:idx_split, :], 
                                       full_df['start_month_scaled'][:idx_split].values.reshape(-1, 1),
                                       full_df['start_hour_scaled'][:idx_split].values.reshape(-1, 1),
                                       full_df['morning_scaled'][:idx_split].values.reshape(-1, 1)])

X_test_hour_scaled = hstack([full_sites_sparse[idx_split:, :], 
                                      full_df['start_month_scaled'][idx_split:].values.reshape(-1, 1),
                                      full_df['start_hour_scaled'][idx_split:].values.reshape(-1, 1)])
X_test_morning_scaled = hstack([full_sites_sparse[idx_split:, :], 
                                      full_df['start_month_scaled'][idx_split:].values.reshape(-1, 1),
                                      full_df['morning_scaled'][idx_split:].values.reshape(-1, 1)])
X_test_hour_morning_scaled = hstack([full_sites_sparse[idx_split:, :], 
                                      full_df['start_month_scaled'][idx_split:].values.reshape(-1, 1),
                                      full_df['start_hour_scaled'][idx_split:].values.reshape(-1, 1),
                                      full_df['morning_scaled'][idx_split:].values.reshape(-1, 1)])

# Вычисляем ROC AUC с использованием новых признаков
roc_auc_with_hour = get_auc_lr_valid(X_train_hour_scaled, y_train)
print(f'ROC AUC на валидационной выборке с признаками start_hour: {roc_auc_with_hour}')

roc_auc_with_morning = get_auc_lr_valid(X_train_morning_scaled, y_train)
print(f'ROC AUC на валидационной выборке с признаками morning: {roc_auc_with_morning}')

roc_auc_with_hour_morning = get_auc_lr_valid(X_train_hour_morning_scaled, y_train)
print(f'ROC AUC на валидационной выборке с признаками start_hour и morning: {roc_auc_with_hour_morning}')

ROC AUC на валидационной выборке с признаками start_hour: 0.9799764792617462
ROC AUC на валидационной выборке с признаками morning: 0.9811914381440888
ROC AUC на валидационной выборке с признаками start_hour и morning: 0.9826369616986874


### 4. Подбор коэффицициента регуляризации

Итак, мы ввели признаки, которые улучшают качество нашей модели по сравнению с первым бейслайном. Можем ли мы добиться большего значения метрики? После того, как мы сформировали обучающую и тестовую выборки, почти всегда имеет смысл подобрать оптимальные гиперпараметры -- характеристики модели, которые не изменяются во время обучения. Например, на 3 неделе вы проходили решающие деревья, глубина дерева это гиперпараметр, а признак, по которому происходит ветвление и его значение -- нет. В используемой нами логистической регрессии веса каждого признака изменяются и во время обучения находится их оптимальные значения, а коэффициент регуляризации остается постоянным. Это тот гиперпараметр, который мы сейчас будем оптимизировать.

Посчитайте качество на отложенной выборке с коэффициентом регуляризации, который по умолчанию `C=1`:

In [16]:
# Ваш код здесь
# Вычисляем ROC AUC с коэффициентом регуляризации по умолчанию
roc_auc_default_C = get_auc_lr_valid(X_train_hour_morning_scaled, y_train)
print(f'ROC AUC на валидационной выборке с коэффициентом регуляризации по умолчанию: {roc_auc_default_C}')

ROC AUC на валидационной выборке с коэффициентом регуляризации по умолчанию: 0.9826369616986874


Постараемся побить этот результат за счет оптимизации коэффициента регуляризации. Возьмем набор возможных значений C и для каждого из них посчитаем значение метрики на отложенной выборке.

Найдите `C` из `np.logspace(-3, 1, 10)`, при котором ROC AUC на отложенной выборке максимален. 

In [17]:
# Ваш код здесь
from sklearn.model_selection import GridSearchCV

param_grid = {'C': np.logspace(-3, 1, 10)}

grid_search = GridSearchCV(LogisticRegression(random_state=17, solver='liblinear'), param_grid, scoring='roc_auc', cv=5)
grid_search.fit(X_train_hour_morning_scaled, y_train)

print(f'Наилучший параметр C: {grid_search.best_params_["C"]}')
print(f'ROC AUC с лучшим параметром: {grid_search.best_score_}')

Наилучший параметр C: 1.2915496650148828
ROC AUC с лучшим параметром: 0.9358561840364488


In [20]:
grid_search.cv_results_

{'mean_fit_time': array([ 0.83303094,  0.8900775 ,  1.08394055,  1.34105601,  1.77407851,
         2.59735155,  3.36430755,  4.97266665,  9.65196848, 16.06995363]),
 'std_fit_time': array([0.07933342, 0.07401204, 0.06315932, 0.10747658, 0.10348259,
        0.465747  , 0.31037696, 0.43041089, 1.10815202, 1.46300785]),
 'mean_score_time': array([0.01738558, 0.01661639, 0.01631136, 0.01560545, 0.01669412,
        0.02041655, 0.02199755, 0.01561327, 0.02436934, 0.01998138]),
 'std_score_time': array([0.00284179, 0.0015049 , 0.00057771, 0.00135006, 0.00195335,
        0.00526341, 0.00458298, 0.00100477, 0.00727392, 0.00357287]),
 'param_C': masked_array(data=[0.001, 0.0027825594022071257, 0.007742636826811269,
                    0.021544346900318832, 0.05994842503189409,
                    0.1668100537200059, 0.46415888336127775,
                    1.2915496650148828, 3.593813663804626, 10.0],
              mask=[False, False, False, False, False, False, False, False,
                   

Наконец, обучите модель с найденным оптимальным значением коэффициента регуляризации и с построенными признаками `start_hour`, `start_month` и `morning`. Если вы все сделали правильно и загрузите это решение, то повторите второй бенчмарк соревнования.

In [19]:
# Ваш код здесь
# Создаем новый логистический регрессор с оптимальным значением C
logit_optimal_C = LogisticRegression(C=grid_search.best_params_['C'], random_state=17, solver='liblinear')

# Обучаем модель с использованием оптимального C и новых признаков
logit_optimal_C.fit(X_train_hour_morning_scaled, y_train)

# Делаем прогноз для тестовой выборки
test_predict_proba_optimal_C = logit_optimal_C.predict_proba(X_test_hour_morning_scaled)[:, 1]

write_to_submission_file(test_predict_proba_optimal_C, 'submission_optimal_C.csv')